### install detectron2

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44090 sha256=8ffe8adc442ff02ecada060f8aa663d097643e8cca05a56cabfd214e806f109e
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.12.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
flax 0.6.9 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatibl

### download pretrained model

In [ ]:
!gdown 1w3LpgFjB_QpJ4tGa0m1QXBd1eXNKwMzv

Downloading...
From: https://drive.google.com/uc?id=1w3LpgFjB_QpJ4tGa0m1QXBd1eXNKwMzv
To: /content/model_final.pth
100% 330M/330M [00:01<00:00, 183MB/s]


### download sample tif image

In [ ]:
# !gdown 13wiEc7fEvkYXmzRxO-_kQiktm0LtTrSu

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#To get road coordiates
!gdown 1YPPjNFnGcmm1spgKPNOR7XnXrUj5LgYA

Downloading...
From: https://drive.google.com/uc?id=1YPPjNFnGcmm1spgKPNOR7XnXrUj5LgYA
To: /content/roads_coord.csv
100% 240k/240k [00:00<00:00, 70.9MB/s]


In [ ]:
#To get VMT iformation
!gdown 1_68WqpY8eV4o01Kw1UaCWJwfgRyDTsOJ

Downloading...
From: https://drive.google.com/uc?id=1_68WqpY8eV4o01Kw1UaCWJwfgRyDTsOJ
To: /content/y2017_vmt.csv
100% 86.4k/86.4k [00:00<00:00, 75.5MB/s]


In [ ]:
#To get zicodes of houston
!gdown 1CN8xNLu_M-WtJ-j1buLu12ztcuIbcWCp

Downloading...
From: https://drive.google.com/uc?id=1CN8xNLu_M-WtJ-j1buLu12ztcuIbcWCp
To: /content/zipcodes_h.csv
100% 5.79k/5.79k [00:00<00:00, 19.4MB/s]


In [ ]:
#To dowload the road images folder
!gdown 1y937TrXG9ifFeYJfjz-vYw_RgmiVtqLi

Downloading...
From: https://drive.google.com/uc?id=1y937TrXG9ifFeYJfjz-vYw_RgmiVtqLi
To: /content/houston_road_images.zip
100% 608M/608M [00:05<00:00, 113MB/s]


In [ ]:
#To unzip the images folder
import zipfile
def unzip_folder(zip_path, destination):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(destination)

unzip_folder('/content/houston_road_images.zip', 'download_images')

In [ ]:
#To create a folder to save outut details
import os
folder_path = "Output_labeled"
os.makedirs(folder_path, exist_ok=True)

# use trained model for car detection

## testing code

In [ ]:

import os
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import matplotlib

# Setup object detector
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
predictor = DefaultPredictor(cfg)

roads_cord=pd.read_csv("/content/roads_coord.csv")
zipcodes=pd.read_csv("/content/zipcodes_h.csv")
vmt= pd.read_csv("/content/y2017_vmt.csv")
deafualt_zip=77002

# Define input and output folders
input_folder = '/content/download_images/content/download_images'
output_folder = '/content/Output_labeled'

# Check if output.csv already exists, if yes read the existing results
if os.path.isfile('output.csv'):
    df_existing = pd.read_csv('output.csv')
    results = df_existing.to_dict('records')
else:
    results = []

# Iterate over all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.tif') or filename.endswith('.png') or filename.endswith('.jpg'):
        # Open the RGBA image
        root = os.path.join(input_folder, filename)
        rgba_image = Image.open(root)

        # Convert the RGBA image to RGB format
        rgb_image = rgba_image.convert('RGB')

        # Save the RGB image as a JPEG file
        rgb_filename = os.path.splitext(filename)[0] + '.jpg'
        rgb_path = os.path.join(output_folder, rgb_filename)
        rgb_image.save(rgb_path)

        # Close the images
        rgba_image.close()
        rgb_image.close()

        # Apply object detection on the image
        img = cv2.imread(rgb_path)
        xy_coord = []
        for r in range(0,img.shape[0],256):
            for c in range(0,img.shape[1],256):
              im = img[r:r+256, c:c+256,:]
              height, width  = im.shape[:2]
              if width == 256 and height == 256:
                pass
              else:
                padding_y = (256-im.shape[0])
                padding_x = (256-im.shape[1])
                im = np.pad(im, ((0,padding_y),(0,padding_x),(0,0)))

              y0,x0 = r, c
              outputs = predictor(im)
              instances = outputs["instances"]
              detected_class_indexes = instances.pred_classes
              prediction_boxes = instances.pred_boxes

              for idx, coordinates in enumerate(prediction_boxes):
                  class_index = detected_class_indexes[idx]
                  xy = coordinates.cpu().numpy()
                  x1 = xy[0] + float(x0)
                  y1 = xy[1] + float(y0)
                  x2 = xy[2] + float(x0)
                  y2 = xy[3] + float(y0)
                  xy_coord.append([x1, y1, x2, y2])


        # Visualize the detected objects
        #printing the number of cars
        #print("car coun : ",len(xy_coord)*2)
        im = Image.open(rgb_path)

        # Acquire default dots per inch value of matplotlib
        dpi = matplotlib.rcParams['figure.dpi']
        # Determine the figures size in inches to fit your image
        im = np.array(im)
        height, width, depth = im.shape
        figsize = width / float(dpi), height / float(dpi)

        fig, ax = plt.subplots(figsize=figsize)

        for xy in xy_coord:
            x1 = xy[0]
            y1 = xy[1]
            width = xy[2]-xy[0]
            height = xy[3]-xy[1]
            rect = patches.Rectangle((x1,y1), width, height, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect) # Add the patch to the Axes

        # Display the image
        ax.imshow(im)


        # Save the labeled image
        labeled_filename = os.path.splitext(filename)[0] + '_labeled.jpg'
        labeled_path = os.path.join(output_folder, labeled_filename)
        fig.savefig(labeled_path, dpi=dpi, bbox_inches='tight')

        # Close the figure
        plt.close(fig)
            # Get the number of detected objects
    num_labels = len(xy_coord)*2
    # Print the number of detected objects
        #print("file: ", filename, " - num labels: ", num_labels)



    row=[]
    zips=[]
    if (filename in roads_cord['filename'].values):
      row=roads_cord.loc[roads_cord['filename'] == filename].values
      zips=zipcodes.loc[((zipcodes['Lat_min']<=row[0][1]) & (zipcodes['Lat_max']>=row[0][2])) & ((zipcodes['Lon_min']<=row[0][3]) & (zipcodes['Lon_max']>=row[0][4])),"Zipcode"].values
    print(filename)
    if len(zips)==0:
        zips=zipcodes.loc[((zipcodes['Lat_min']<=row[0][1]) & (zipcodes['Lat_max']>=row[0][2])),"Zipcode"].values
        if len(zips)==0:
            zips=zipcodes.loc[((zipcodes['Lon_min']<=row[0][3]) & (zipcodes['Lon_max']>=row[0][4])),"Zipcode"].values
            if len(zips)==0:
                zips=[deafualt_zip]
        zc=zips[0]
    else:
        zc=zips[0]
    # print(zc)

    vt=vmt.loc[vmt['zipcode']==zc,"VMT_Cumulative"].mean()
        # Append the result to the list
    results.append({'file_name': filename, 'num_labels': num_labels, 'zipcode': zc,'VMT':vt})
        # Create a dataframe from the results list
    df = pd.DataFrame(results)
        # Write the dataframe to the output file
    df.to_csv('output.csv', index=False)




  pixel_mean
  pixel_std
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


IH0010-XG.pkl_781.64_782.14_1046.tif
IH0069-XG.pkl_25.2_25.7_547.tif
FM0762-KG.pkl_1.5_2.0_1155.tif
IH0010-XG.pkl_748.655_749.155_328.tif
FM3005-RG.pkl_7.983_8.483_762.tif
PR0066-KG.pkl_0.0_0.5_57.tif
SH0035-LG.pkl_52.073_52.573_2280.tif
IH0010-XG.pkl_789.659_790.159_2347.tif
IH0010-XG.pkl_746.199_746.655_175.tif
UA0090-LG.pkl_175.54_176.154_1224.tif
FM1495-KG.pkl_2.414_2.914_855.tif
IH0010-LG.pkl_767.14_767.616_1728.tif
IH0010-RG.pkl_792.697_793.197_3101.tif
US0290-LG.pkl_261.647_262.147_348.tif
SH0035-LG.pkl_50.622_50.922_2200.tif
SL0008-AG.pkl_62.563_63.063_1070.tif
IH0010-LG.pkl_792.161_792.661_3110.tif
IH0610-RG.pkl_8.995_9.495_624.tif
IH0069-RG.pkl_27.189_27.689_725.tif
IH0010-AG.pkl_776.202_776.71_1915.tif
IH0069-RG.pkl_36.896_37.396_2687.tif
FM0521-KG.pkl_43.019_43.519_1538.tif
US0290-XG.pkl_256.982_257.082_2454.tif
IH0010-XG.pkl_747.656_748.156_2732.tif
IH0610-XG.pkl_11.901_12.001_282.tif
IH0069-AG.pkl_8.223_8.7_2626.tif
FM0762-KG.pkl_21.215_21.328_1024.tif
IH0610-RG.pkl_11.60

- Output is saved in .csv file